In [6]:
pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 715.4/715.4 kB 15.3 MB/s eta 0:00:00


In [7]:
import os
import shutil
import random
!pip install tqdm --upgrade
from tqdm.notebook import tqdm

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"

In [8]:
import os
import shutil
import random
from tqdm import tqdm

def train_test_split(path, neg_path=None, split=0.2):
    print("------ PROCESS STARTED -------")

    files = list(set([name[:-4] for name in os.listdir(path)]))  # removing duplicate names i.e. counting only number of images

    print(f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size
    train_path_img = "train/images"
    train_path_label = "train/labels"
    val_path_img = "val/images"
    val_path_label = "val/labels"
    os.makedirs(train_path_img, exist_ok=True)
    os.makedirs(train_path_label, exist_ok=True)
    os.makedirs(val_path_img, exist_ok=True)
    os.makedirs(val_path_label, exist_ok=True)

    for filex in tqdm(files[:train_size]):
        if filex == 'classes':
            continue
        shutil.copy2(path + filex + '.jpg', f"{train_path_img}/" + filex + '.jpg')
        shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')
    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)]))  # removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path + filex + ".jpg", f"{train_path_img}/" + filex + '.jpg')
        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")
        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")

    for filex in tqdm(files[train_size:]):
        if filex == 'classes':
            continue
        shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg')
        shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

train_test_split('/content/drive/MyDrive/yolov8/data/')  # without negative images


------ PROCESS STARTED -------
--- This folder has a total number of 66 images---


100%|██████████| 53/53 [00:06<00:00,  8.29it/s]


------ Training data created with 80% split 53 images -------


100%|██████████| 13/13 [00:00<00:00, 18.37it/s]


In [9]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.16 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.5/78.2 GB disk)


In [10]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/yolov8/dataset.yaml epochs=10 imgsz=640 batch=8 project=/content/drive/MyDrive/yolov8/training_results name=football


100% 21.5M/21.5M [00:00<00:00, 251MB/s]
Ultralytics YOLOv8.1.16 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/yolov8/dataset.yaml, epochs=10, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/yolov8/training_results, name=football7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fr

In [21]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/yolov8/training_results/football7/weights/best.pt conf=0.55 source=/content/drive/MyDrive/yolov8/test_images


Ultralytics YOLOv8.1.16 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

image 1/7 /content/drive/MyDrive/yolov8/test_images/Copy of 6ty_f_110_54r.jpg: 384x640 12 players, 1 football, 67.5ms
image 2/7 /content/drive/MyDrive/yolov8/test_images/Copy of 6ty_f_130_22d.jpg: 384x640 16 players, 10.7ms
image 3/7 /content/drive/MyDrive/yolov8/test_images/Copy of 6ty_f_50_we.jpg: 384x640 12 players, 15.0ms
image 4/7 /content/drive/MyDrive/yolov8/test_images/Copy of 6ty_f_65_we.jpg: 384x640 12 players, 10.9ms
image 5/7 /content/drive/MyDrive/yolov8/test_images/Copy of 6ty_f_80_54r.jpg: 384x640 12 players, 10.7ms
image 6/7 /content/drive/MyDrive/yolov8/test_images/Copy of 6ty_f_85.jpg: 384x640 8 players, 10.7ms
image 7/7 /content/drive/MyDrive/yolov8/test_images/Copy of 6ty_f_90_we.jpg: 384x640 12 players, 10.7ms
Speed: 2.5ms preprocess, 19.5ms inference, 101.5ms postprocess per image at shape (1, 3, 3

In [22]:
!cp -r /content/runs/detect/predict /content/drive/MyDrive/yolov8/output